In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from random import shuffle
from PIL import Image
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras import regularizers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_size = 12000
test_size = 4000

image_size = 256
block_size = 32 #256/8
input_shape = (32,32,3)
batch_size = 64

reg_constant = 0.0001
learning_rate = 0.01
momentum = 0.9

pieces = 'prbnkqPRBNKQ'
numbers = '12345678'

In [ ]:
train = glob.glob("../input/dataset/train/*.jpeg")
test = glob.glob("../input/dataset/test/*.jpeg")

shuffle(train)
shuffle(test)

train = train[:train_size]
test = test[:test_size]

In [ ]:
def get_fen(fileLocation):
    fileName = os.path.basename(fileLocation)
    return os.path.splitext(fileName)[0]

In [ ]:
print(get_fen(train[0]))
print(get_fen(train[1]))
print(get_fen(train[2]))

In [ ]:
#12 pieces, need to one hot encode a the board with 13 possible values
def one_hot_encode(fen):
    fen = fen.replace("-", "")
    
    eye = np.eye(13)
    out = np.zeros((0, 13))
    
    for char in fen:
        if (char in numbers):
            toAppend = np.tile(eye[12], (int(char), 1))
        else:
            piece_index = pieces.index(char)
            toAppend =  eye[piece_index].reshape((1, 13))
        out = np.append(out, toAppend, axis=0)
    
    return out
        

In [ ]:
#input is 64 by 13
def one_hot_decode(one_hot):
    #one_hot = np.resize(one_hot, (8,8,13))
    out = ''
    for i in range(8):
        for j in range(8):
            #eyeInv, = np.where(one_hot[i][j]==1)[0]
            eyeInv = one_hot[i][j]
            if (eyeInv == 12):
                out += ' '
            else:
                out += pieces[eyeInv]
        if (i<7):
            out += '-'
    
    for i in range(8, 0, -1):
        out = out.replace(i*' ', str(i))
    
    return out

In [ ]:
def generate_image_data(image_location):
    image = np.array(Image.open(image_location).resize([image_size,image_size]))/255
    out = np.zeros((64, block_size, block_size, 3))
    for i in range(8):
        for j in range(8):
            out[i*8 + j] = image[i*block_size:(i+1)*block_size,j*block_size:(j+1)*block_size]
    
    return out

In [ ]:
def generate_training_data(data):
    for i in range(len(data)):
        image_location = data[i]
        fen = get_fen(image_location)
        x = generate_image_data(image_location)
        y = one_hot_encode(fen)
    
        yield x, y

In [ ]:
next(generate_prediction(train))[0].shape

In [ ]:
def generate_training_data(data, batch_size):
    for i in range(len(data)//batch_size):
        x = np.zeros((batch_size, 64, block_size, block_size, 3))
        y = np.zeros((batch_size, 64, 13))
        for j in range(batch_size): 
            image_location = data[i*batch_size + j]
            fen = get_fen(image_location)
            x[j] = generate_image_data(image_location)
            y[j] = one_hot_encode(fen)
    
        yield x, y

In [ ]:
def generate_prediction(data):
    for i in range(len(data)):
        image_location = data[i]
        yield generate_image_data(image_location)

In [ ]:
def generate_prediction(data, batch_size):
    for i in range(len(data)//batch_size):
        prediction = np.zeros((batch_size, 64, block_size, block_size, 3))
        for j in range(batch_size): 
            image_location = data[i*batch_size + j]
            prediction[j] = generate_image_data(image_location)
    
        yield prediction

In [ ]:
#build model
a = Input(shape=(input_shape))
x = Conv2D(16, [3, 3], padding='same', bias_initializer='random_uniform')(a)
x = LeakyReLU()(x)
x = Conv2D(32, [3, 3], padding='same', bias_initializer='random_uniform')(x)
x = LeakyReLU()(x)
x = Conv2D(64, [3, 3], padding='same', bias_initializer='random_uniform')(x)
x = LeakyReLU()(x)
x = Flatten()(x)
x = Dense(128, bias_initializer='random_uniform')(x)
x = LeakyReLU()(x)
x = Dropout(0.4)(x)
x = Dense(13, activation='softmax', bias_initializer= 'random_uniform')(x)

model = Model(inputs = [a], outputs = [x])
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr = learning_rate, momentum = momentum),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(generate_training_data(train), steps_per_epoch=train_size)

In [ ]:
predictions = (model.predict_generator(generate_prediction(test), len(test)))
predictions_processed = predictions.reshape(-1, 8, 8, 13).argmax(axis=3)
fen_predictions = np.array([one_hot_decode(y) for y in predictions_processed])
fen_real = np.array([get_fen(y_real) for y_real in test])

acc = (fen_predictions == fen_real).astype(float).mean()

print("Accuracy is: " + str(acc))